In [ ]:
!pip install opencv-python

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt



# DEF. PARAMETERS
img_row, img_column = 300, 300
target_size = (img_row, img_column)
num_channel = 3
num_batch = 1
# include the path containing the model (.lite, .tflite)
#model_path = "final_model.tflite"
model_path = "/content/drive/My Drive/final_model1.tflite"


input_img_name = '/content/4.jpg'#'front_image.png'
image_orig = cv2.imread(input_img_name)

shape = image_orig.shape
H = shape[0]
W = shape[1]

if(H>W):
 top =0
 bottom=0
 left= int((H-W)/2)
 right = left
else:
 top = int((W-H)/2)
 bottom = top
 left= 0
 right = 0

image_padded = image_orig.copy()
image_padded = cv2.copyMakeBorder(image_padded,top, bottom, left, right ,cv2.BORDER_CONSTANT,0)


image = cv2.resize(image_padded, target_size)

# for float inpu model
inp_image = np.zeros((1,target_size[1], target_size[0], 3), dtype=np.float32)
inp_image[0,:,:,:] = image[:,:,0:3].astype(np.float32)


# for uint8
# inp_image = np.zeros((1,target_size[1], target_size[0], 3), dtype=np.uint8)
# inp_image[0,:,:,:] = image[:,:,0:3].astype(np.uint8)


# TFLITE INTERPRETER CON.
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
# obtaining the input-output shapes and types
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

#print the details and idex of your inputs and outputs
print(input_details) # you will see that the index is = 338, this will be needed if you use C++ or jave tflite api

# input_details
# [{'dtype': numpy.float32,
#   'index': 338,
#   'name': 'normalized_input_image_tensor',
#   'quantization': (0.0, 0L),
#   'shape': array([  1, 300, 300,   3], dtype=int32)}]


print(output_details)  # you will see that the index is = 330, 331, 332, 333, this will be needed if you use C++ or jave tflite api


# output_details

# [{'dtype': numpy.float32,
#   'index': 330,
#   'name': 'TFLite_Detection_PostProcess',
#   'quantization': (0.0, 0L),
#   'shape': array([ 1, 10,  4], dtype=int32)},
#  {'dtype': numpy.float32,
#   'index': 331,
#   'name': 'TFLite_Detection_PostProcess:1',
#   'quantization': (0.0, 0L),
#   'shape': array([ 1, 10], dtype=int32)},
#  {'dtype': numpy.float32,
#   'index': 332,
#   'name': 'TFLite_Detection_PostProcess:2',
#   'quantization': (0.0, 0L),
#   'shape': array([ 1, 10], dtype=int32)},
#  {'dtype': numpy.float32,
#   'index': 333,
#   'name': 'TFLite_Detection_PostProcess:3',
#   'quantization': (0.0, 0L),
#   'shape': array([1], dtype=int32)}]


net_input = inp_image.reshape(input_details[0]['shape'])

# RUNNING INTERPRETER
# setting the input tensor with the selected input
interpreter.set_tensor(input_details[0]['index'], net_input)

# running inference
interpreter.invoke()

print('====================================================================')
net_out_0 = interpreter.get_tensor(output_details[0]['index']) # 'TFLite_Detection_PostProcess' 
print(net_out_0)

print('====================================================================')
net_out_1 = interpreter.get_tensor(output_details[1]['index']) # 'TFLite_Detection_PostProcess:1'
print(net_out_1)

print('====================================================================')
net_out_2 = interpreter.get_tensor(output_details[2]['index']) # 'TFLite_Detection_PostProcess:2'
print(net_out_2)

print('====================================================================')
net_out_3 = interpreter.get_tensor(output_details[3]['index']) # 'TFLite_Detection_PostProcess:3'
print(net_out_3)


[{'name': 'normalized_input_image_tensor', 'index': 256, 'shape': array([  1, 300, 300,   3], dtype=int32), 'shape_signature': array([  1, 300, 300,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'TFLite_Detection_PostProcess', 'index': 248, 'shape': array([ 1, 10,  4], dtype=int32), 'shape_signature': array([ 1, 10,  4], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'TFLite_Detection_PostProcess:1', 'index': 249, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([ 1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': 